<a href="https://colab.research.google.com/github/EtzionR/LM4GeoAI/blob/main/Tutorial_Cohere_and_MCP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial: Basic use of cohere and MCP

### created by Etzion Harari | Geo-AI Course

[**https://github.com/EtzionR/LM4GeoAI**](https://github.com/EtzionR/LM4GeoAI)

## pip install cohere

In [11]:
!pip install -q cohere

## imports

In [12]:
from getpass import getpass
import cohere

## Enter cohere API key

You can get your cohere API Key here (you should log-in / sign-in to cohere):

[https://dashboard.cohere.com/api-keys](https://dashboard.cohere.com/api-keys)

In [13]:
KEY = getpass("Please Enter COHERE KEY:\n")

len(KEY)

Please Enter COHERE KEY:
··········


40

## Init cohere Client

In [14]:
client = cohere.Client(api_key=KEY)
client

## Example of calling to cohere model

In [15]:
QUERY = 'What is panda bear?' # an example of a simple query
MODEL  = "command-a-03-2025"
TEMP = .01

output = client.chat(model=MODEL,
                     message=QUERY,
                     temperature=TEMP).text

print(output)

A panda bear, often simply referred to as a panda, is a species of bear native to central China. Scientifically known as *Ailuropoda melanoleuca*, the panda is one of the most recognizable and beloved animals in the world, primarily due to its distinctive black and white coat. Here are some key facts about panda bears:

1. **Physical Characteristics**:  
   - Pandas have a stocky body, round face, and black patches around their eyes, ears, and across their body.  
   - Adults typically weigh between 70 to 125 kilograms (150 to 275 pounds) and stand about 75 to 80 centimeters (30 inches) tall at the shoulder.  

2. **Diet**:  
   - Pandas are primarily herbivores, with bamboo making up 99% of their diet. They may also eat other vegetation, small animals, and fish occasionally.  
   - An adult panda can eat up to 12–38 kilograms (26–84 pounds) of bamboo daily.  

3. **Habitat**:  
   - Pandas are found in temperate forests in the mountainous regions of central China, particularly in Sich

## Example MCP

In [16]:
# system prompt
system_prompt = """You are an AI assistant with access to the following tools:
calculator tool, which enable you to perform simple calculations"""

# tools schema
tools = [
    {
        "name": "calculator",
        "description": "can perform simple caluclations",
        "parameter_definitions": {
            "a": {
                "description": "first number",
                "type": "float",
                "required": True
            },
            "b": {
                "description": "second number",
                "type": "float",
                "required": True
            },
            "operation": {
                "description": "calculation operator ('+', '-', '*', '/')",
                "type": "str",
                "required": False
            }
        }
    }]

# calculator function
def calculator(a, b, operation):
    if operation == '+':
        return a + b
    elif operation == '-':
        return a - b
    elif operation == '*':
        return a * b
    elif operation == '/':
        return a / b

# tools map
tools_map = {calculator.__name__: calculator}
tools_map

{'calculator': <function __main__.calculator(a, b, operation)>}

## Use simple cohere model:

In [17]:
A = 727.3467
B = 167.1788

QUERY = f'can you caluclate this: {A}*{B}=? (please return only the calculation output!)'
MODEL  = "command-a-03-2025"

output_without_mcp = client.chat(model=MODEL,
                                 message=QUERY).text

print(f'Output without MCP:\n{output_without_mcp}')

Output without MCP:
418.25776296


## Use the same cohere model with MCP

In [18]:
response = client.chat(
    model = MODEL,
    message = QUERY,
    preamble = system_prompt,
    tools = tools
)

output_with_mcp = tools_map[response.tool_calls[0].name](**response.tool_calls[0].parameters)

print(f'Model output: {response.text}\n\nOutput with MCP:\n{output_with_mcp}')

Model output: I will use the calculator tool to multiply 24.3467 by 17.1788.

Output with MCP:
418.2470899599999


## Comparing the both answers

In [20]:
MSG1 = 'The model without MCP perform better'
MSG2 = 'The model with MCP perform better'

ACTUAL_ANSWER = A*B

output_msg = MSG2 if abs(ACTUAL_ANSWER-float(output_with_mcp))<abs(ACTUAL_ANSWER-float(output_without_mcp)) else MSG1

print(output_msg)


The model with MCP perform better


Create by Etzion Harari | Geo-AI Course | [https://github.com/EtzionR/LM4GeoAI](https://github.com/EtzionR/LM4GeoAI)